>Data Preprocessing

In [6]:
import os
import xarray as xr
import pandas as pd
from tqdm import tqdm

# ✅ Set your base data path
base_path = "F:/projects/personal project/data/Atmospheric_variable"

# ✅ Map custom variable names to their folder and internal NetCDF variable name
variable_folders = {
    "HGT": ("IMDAA_HGT_prl_1.08_1990_2020", "HGT_prl"),
    "RH": ("IMDAA_RH_prl_1.08_1990_2020", "RH_prl"),
    "TMP": ("IMDAA_TMP_prl_1.08_1990_2020", "TMP_prl"),
    "UGRD": ("IMDAA_UGRD_prl_1.08_1990_2020", "UGRD_prl"),
    "VGRD": ("IMDAA_VGRD_prl_1.08_1990_2020", "VGRD_prl")
}

def load_variable(nc_var_name, folder_name, level=0):
    """Load and process a NetCDF variable"""
    folder_path = os.path.join(base_path, folder_name)
    all_years = []

    for fname in tqdm(sorted(os.listdir(folder_path)), desc=f"Loading {nc_var_name}"):
        if fname.endswith(".nc"):
            fpath = os.path.join(folder_path, fname)
            try:
                ds = xr.open_dataset(fpath)

                # Rename if needed
                if 'lat' in ds.dims: ds = ds.rename({'lat': 'latitude'})
                if 'lon' in ds.dims: ds = ds.rename({'lon': 'longitude'})

                data = ds[nc_var_name]

                # Select one pressure level
                if 'level' in data.dims:
                    data = data.isel(level=level)

                # Spatial average
                spatial_avg = data.mean(dim=["latitude", "longitude"], skipna=True)

                # Daily average
                daily_avg = spatial_avg.resample(time='1D').mean()

                df = daily_avg.to_dataframe(name=nc_var_name)
                all_years.append(df)
                ds.close()

            except Exception as e:
                print(f"⚠️ Error reading {fpath}: {e}")

    return pd.concat(all_years)

# ✅ Load and combine all variables
df_all = None

for custom_name, (folder, nc_var_name) in variable_folders.items():
    df = load_variable(nc_var_name, folder)
    df.rename(columns={nc_var_name: custom_name}, inplace=True)
    if df_all is None:
        df_all = df
    else:
        df_all = df_all.join(df, how="outer")

# Drop missing values
df_all.dropna(inplace=True)

# Save
df_all.to_csv("F:/projects/personal project/weather prediction_v1/data/weather_atmospheric_processed.csv")
print("✅ All atmospheric variables processed and saved to CSV!")


Loading VGRD_prl:  52%|█████████████████████████████████                               | 16/31 [00:03<00:03,  4.92it/s]C:\Users\karma\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:88: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
Loading VGRD_prl: 100%|████████████████████████████████████████████████████████████████| 31/31 [00:06<00:00,  4.88it/s]


✅ All atmospheric variables processed and saved to CSV!


In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# 1. Load dataset
df = pd.read_csv("F:/projects/personal project/weather prediction_v1/data/weather_atmospheric_processed.csv", index_col=0, parse_dates=True)

# 2. Shift TMP to create prediction target
df["TMP_target"] = df["TMP"].shift(-1)

# 3. Drop rows with NaN, inf, or -inf
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.dropna(inplace=True)

# 4. Feature set and target
features = ["HGT", "RH", "TMP", "UGRD", "VGRD"]
X = df[features]
y = df["TMP_target"]

# 5. Split into train/test (chronologically)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, shuffle=False, test_size=0.2
)

# 6. Scale inputs using MinMaxScaler
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# ✅ Sanity check
print("✅ No NaN or Inf values in training/testing sets.")


✅ No NaN or Inf values in training/testing sets.


In [10]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train_scaled, y_train)
rf_preds = rf.predict(X_test_scaled)


In [12]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Reshape for LSTM: (samples, timesteps, features)
X_train_lstm = X_train_scaled.reshape((X_train_scaled.shape[0], 1, X_train_scaled.shape[1]))
X_test_lstm = X_test_scaled.reshape((X_test_scaled.shape[0], 1, X_test_scaled.shape[1]))

# Build LSTM model
lstm = Sequential()
lstm.add(LSTM(50, input_shape=(1, X_train_scaled.shape[1])))
lstm.add(Dense(1))
lstm.compile(optimizer='adam', loss='mse')

# Train
lstm.fit(X_train_lstm, y_train, epochs=20, batch_size=32, verbose=1)
lstm_preds = lstm.predict(X_test_lstm).flatten()


C:\Users\karma\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/20
3680/3680 ━━━━━━━━━━━━━━━━━━━━ 7s 1ms/step - loss: 45046.2969
Epoch 2/20
3680/3680 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - loss: 6985.0542
Epoch 3/20
3680/3680 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - loss: 1138.1589
Epoch 4/20
3680/3680 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - loss: 465.1202
Epoch 5/20
3680/3680 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - loss: 461.4921
Epoch 6/20
3680/3680 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - loss: 454.2203
Epoch 7/20
3680/3680 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - loss: 456.7516
Epoch 8/20
3680/3680 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - loss: 451.5705
Epoch 9/20
3680/3680 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - loss: 408.9851
Epoch 10/20
3680/3680 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - loss: 374.6411
Epoch 11/20
3680/3680 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - loss: 329.9389
Epoch 12/20
3680/3680 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - loss: 274.0384
Epoch 13/20
3680/3680 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - loss: 226.0427
Epoch 14/20
3680/3680 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - loss: 180.25

In [13]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

def evaluate_model(name, y_true, y_pred):
    print(f"\n📊 {name} Evaluation:")
    print(f"MAE: {mean_absolute_error(y_true, y_pred):.4f}")
    print(f"RMSE: {np.sqrt(mean_squared_error(y_true, y_pred)):.4f}")
    print(f"R² Score: {r2_score(y_true, y_pred):.4f}")

evaluate_model("Random Forest", y_test, rf_preds)
evaluate_model("LSTM", y_test, lstm_preds)



📊 Random Forest Evaluation:
MAE: 0.5277
RMSE: 0.8319
R² Score: 0.9994

📊 LSTM Evaluation:
MAE: 1.3916
RMSE: 2.0965
R² Score: 0.9964
